In [1]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import random
import numpy as np
device = torch.device('cuda')
from torch.utils.data import default_collate
from torchvision.transforms import v2

### Import Fashion MNIST Dataset

In [2]:
# Download and load the MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform= ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

### Set Seed

In [3]:
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

### Dataloaders and Mixup and Cutmix

In [4]:
batch_size = 64
cutmix = v2.CutMix(num_classes=10)
mixup = v2.MixUp(num_classes=10)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])
def collate_fn(batch):
    return cutmix_or_mixup(*default_collate(batch))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle=False)

### Define LeNet5 CNN with Residual Block architecture

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.stride != 1 or x.shape[1] != out.shape[1]:
            residual = nn.Conv2d(x.shape[1], out.shape[1], kernel_size=1, stride=self.stride).to(device)(x)

        out += residual
        out = self.relu(out)

        return out

class ModifiedLeNet5(nn.Module):
    def __init__(self, input_shape=(1, 28, 28), num_classes=10):
        super(ModifiedLeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            ResidualBlock(32, 64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(2304, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [6]:
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda')
model = ModifiedLeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train Function

In [7]:
def train(train_loader, model, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        # Calculate average training loss for the epoch
        avg_train_loss = train_loss / len(train_dataloader)
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}')

### Test Function

In [8]:
def test(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy for the epoch
    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = 100 * (correct / total)

    return avg_test_loss, accuracy

### Commence Training

In [9]:
train(train_dataloader, model, 50)

Epoch 1/50, Train Loss: 1.1419
Epoch 2/50, Train Loss: 0.9764
Epoch 3/50, Train Loss: 0.9260
Epoch 4/50, Train Loss: 0.9078
Epoch 5/50, Train Loss: 0.8902
Epoch 6/50, Train Loss: 0.8689
Epoch 7/50, Train Loss: 0.8685
Epoch 8/50, Train Loss: 0.8406
Epoch 9/50, Train Loss: 0.8332
Epoch 10/50, Train Loss: 0.8245
Epoch 11/50, Train Loss: 0.8212
Epoch 12/50, Train Loss: 0.8102
Epoch 13/50, Train Loss: 0.7890
Epoch 14/50, Train Loss: 0.7983
Epoch 15/50, Train Loss: 0.7899
Epoch 16/50, Train Loss: 0.7903
Epoch 17/50, Train Loss: 0.7928
Epoch 18/50, Train Loss: 0.7778
Epoch 19/50, Train Loss: 0.7757
Epoch 20/50, Train Loss: 0.7829
Epoch 21/50, Train Loss: 0.7808
Epoch 22/50, Train Loss: 0.7771
Epoch 23/50, Train Loss: 0.7577
Epoch 24/50, Train Loss: 0.7574
Epoch 25/50, Train Loss: 0.7710
Epoch 26/50, Train Loss: 0.7511
Epoch 27/50, Train Loss: 0.7628
Epoch 28/50, Train Loss: 0.7493
Epoch 29/50, Train Loss: 0.7484
Epoch 30/50, Train Loss: 0.7493
Epoch 31/50, Train Loss: 0.7420
Epoch 32/50, Trai

### Test trained model on unseen data

In [10]:
avg_test_loss, acc = test(test_dataloader, model)

In [11]:
print("Accuracy on base model: " + str(acc) + "%")

Accuracy on base model: 93.01%
